# 수정중입니다.

# MADDPG

ML2에서 자체 개발한 `rl2`라이브러리를 사용하여 학습 해보겠습니다.

rl2의 자세한 사용법은 https://github.com/kc-ml2/rl2 를 참고 부탁드립니다.

## 셋업

In [1]:
## 패키지 설치

In [2]:
!pip3 intsall marlenv rl2

ERROR: unknown command "intsall" - maybe you meant "install"


In [3]:
## 패키지 로드

In [4]:
%load_ext tensorboard

In [5]:
import sys
from pprint import pprint

### 학습 디바이스 세팅

In [6]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

## Env

게임 세팅에 적합한 initialization을 해주는 helper factory 메소드인 `make_snake`를 통해 env를 초기화 합니다.

e.g. `n_env` > 1에 대하여 gym의 VecEnv와 동일한 개념의 vectorized env로 초기화가 됩니다.

자세한 사용법은 https://github.com/kc-ml2/marlenv 를 참고 하세요!

In [7]:
from marlenv.wrappers import make_snake

학습할 환경의 세팅은 아래와 같습니다.

PPO는 vectorized env에서 학습이 잘 되는 것으로 알려 져있기 떄문에, `n_env=64`로 세팅합니다.

In [8]:
env_config = dict(
    num_snakes=3,
    n_env=64,
    height=20,
    width=20,
    frame_stack=2,
    vision_range=5,
)

In [9]:
env, obs_shape, ac_shape, props = make_snake(**env_config)

`props` 딕셔너리는 초기화 변수에 따른 환경 세팅의 특성들을 담고 있습니다.

`reorder`는 tensorflow와 torch의 컨볼루션 레이어가 받는 shape의 순서가 다르기 때문에, 재배열을 해주는 파라미터입니다.

gym 컨벤션을 따라서, default값이 False(tensorflow)입니다.

p.s. `tensorflow`는 `(N, H, W, C)`, `torch`는 `(N, C, H, W)`

In [10]:
pprint(list(props.keys()))

['high', 'low', 'n_env', 'num_snakes', 'reorder', 'discrete', 'action_info']


## Config

In [11]:
config = dict(
    batch_size=512, 
    epoch=4, 
    train_interval=128, 
    log_level=10,
    log_interval=5e4, 
    save_interval=1e6, 
    lr=1e-4, 
    gamma=0.99,
    grad_clip=10,
)

In [12]:
# TODO: remove dependency on easydict
from easydict import EasyDict
config=EasyDict(config)

## Model

`rl2`의 predefined model인 `PPOModel` 클래스를 사용하여 학습하겠습니다.

In [13]:
from rl2.agents.ppo import PPOModel

In [14]:
model = PPOModel(
    obs_shape,
    ac_shape,
    recurrent=False,
    discrete=True,
    reorder=props['reorder'],
    optimizer='torch.optim.RMSprop',
    high=props['high']
)

## Agent

`rl2`의 predefined model인 `PPOAgent` 클래스를 사용하여 학습하겠습니다.

In [15]:
from rl2.agents.ppo import PPOAgent

In [16]:
def ppo_agent():
    agent = PPOAgent(
        model,
        train_interval=config['train_interval'],
        n_env=env_config['n_env'],
        batch_size=config['batch_size'],
        num_epochs=config['epoch'],
        buffer_kwargs={
            'size': config['train_interval'],
            'n_env': env_config['n_env'],
        }
    )
    
    return agent

In [17]:
agents = [ppo_agent() for _ in range(env_config['num_snakes'])]

## Train

Agent와 Env의 interaction을 진행시켜주는 Worker 클래스를 사용하여 학습을 진행합니다.

`MaxStepWorker`는 클래스명처럼, 주어진 step 횟수만큼 interaction(rollout)을 진행시키고 종료 합니다. 에피소드 단위로 진행하고 싶으시면 `EpisodicWorker` 메뉴얼을 참고 부탁드립니다.

Single Agent일때 사용하는 `MaxStepWorker`의 MA버전인 `MAMaxStepWorker`를 사용합니다.

In [18]:
from rl2.workers.multi_agent import MAMaxStepWorker

### Logger

`rl2`에서는 본인이 원하는 custom logger 클래스를 정의하여 로깅을 할 수 있습니다.

예시로 제공하는 `Logger` 클래스를 활용하여 로깅을 해보겠습니다.

해당 클래스는 tensorboard의 `FileWriter`객체를 가지고 있습니다.

In [19]:
from rl2.examples.temp_logger import Logger

로깅 componenets
* checkpoint
* tensorboard summary data
* env setting
* etc

In [20]:
config['log_dir'] = './MAPPO'
logger = Logger(name='MAPPO', args=config)

In [21]:
worker = MAMaxStepWorker(
    env, 
    props['n_env'], 
    agents,
    max_steps=int(1e7),
    training=True,
    logger=logger,
    log_interval=config['log_interval'],
    render=True,
    render_interval=int(5e5),
    is_save=True,
    save_interval=config['save_interval'],
)

In [ ]:
worker.run()

## 텐서보드

In [ ]:
# %tensorboard --logdir .

## 에이전트 검증

## 에이전트 제출